<a href="https://colab.research.google.com/github/christ-bot/christ-bot/blob/main/vocabularymatching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spacy


In [ ]:
pip install thefuzz

In [ ]:
#import libralies
import pandas as pd

import nltk
from nltk.corpus import wordnet
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer

import spacy
from spacy.matcher import Matcher

from thefuzz import fuzz, process


# Data

In [ ]:
# set pandas to display all rows
pd.set_option('display.max_rows', None)

In [ ]:
# Matched examples
df_matched = pd.read_csv('matched_data.csv')
df_matched

# Fuzzy string matching

In [ ]:
lemmatizer = WordNetLemmatizer()
stemmer = LancasterStemmer()

In [ ]:
def root_words(string):
    wrds = nltk.word_tokenize(string)
    roots = [lemmatizer.lemmatize(word.lower()) for word in wrds]
    #roots = [stemmer.stem(word.lower()) for word in wrds]
    return roots
root_words('computer')

In [ ]:
def get_synonyms(string):
    synonyms = []
    for syn in wordnet.synsets(string):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    res = []
    for syn in synonyms:
        if '_' in syn:
            res.extend(syn.split('_'))
            #res.append(' '.join(syn.split('_')))
        else:
            res.append(syn)
    return res
get_synonyms('computer')

In [ ]:
def get_match(source_1, source_2):
    def get_ratios(i, j):
        print(f"'{i}' vs '{j}'")

        ratio = fuzz.ratio(i, j)
        print(f"Ratio : {ratio}")

        partial_ratio = fuzz.partial_ratio(root_words(i)[0], j)
        print(f"Partial Ratio : {partial_ratio}")

        token_sort_ratio = fuzz.token_sort_ratio(i, j)
        print(f"Token sort Ratio : {token_sort_ratio}")

        token_set_ratio = fuzz.token_set_ratio(i, j)
        print(f"Token set Ratio : {token_set_ratio}")
        print()

        return partial_ratio, token_set_ratio

    for i in source_2:
        i = i.strip()
        synonyms = get_synonyms(i) if ' ' not in i else None
        all_synonyms = ''
        if synonyms:
            synonym_str = ' '.join(synonyms)
            all_synonyms = i + ' ' + synonym_str

        for j in source_1:
            token_set_ratio = get_ratios(i, j)[1]
            if token_set_ratio == 100:
                break
            if len(all_synonyms) > 0:
                partial_ratio = get_ratios(all_synonyms, j)[0]
                if partial_ratio > 80:
                    break
            else:
                continue
                    

In [ ]:
src_1 = df_matched.source_1
src_2 = df_matched.source_2
get_match(src_1, src_2)